In [12]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

# from tensorflow.keras.layers import LeakyReLU
# from keras.callbacks import EarlyStopping
# import keras

#  Import and read the charity_data.csv.
# import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [13]:
# application_df = application_df.drop(columns=["EIN"])
application_df = application_df.drop(columns=["EIN", "NAME"])
# application_df=pd.get_dummies(application_df, drop_first=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [14]:
# Adjusting some colums
application_counts = application_df["APPLICATION_TYPE"].value_counts()
application_types_to_replace = list(application_counts[application_counts <500].index)
application_df["APPLICATION_TYPE"]=application_df["APPLICATION_TYPE"].replace(application_types_to_replace, "Other")

classification_counts = application_df["CLASSIFICATION"].value_counts()
classifications_to_replace = list(classification_counts[classification_counts <1000].index)
application_df["CLASSIFICATION"]=application_df["CLASSIFICATION"].replace(classifications_to_replace, "Other")

In [15]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [16]:
# Reuse the Split for our preprocessed data into our features and target arrays
X= application_df.drop(columns=["IS_SUCCESSFUL"])
y= application_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
#X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#input_features = X_train_scaled.shape[1]
#nn_2 = tf.keras.models.Sequential()
nn_2 = Sequential()

# First hidden layer
#nn_2.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=input_features))
#nn_2.add(tf.keras.layers.Dense(units=128))
nn_2.add(Dense(units=256, activation='tanh'))
nn_2.add(Dropout(0.2))

# Second hidden layer
#nn_2.add(LeakyReLU(alpha=0.1))
#nn_2.add(tf.keras.layers.Dense(units=64, activation='relu'))
nn_2.add(Dense(units=128, activation='tanh'))
nn_2.add(Dropout(0.2))

# Therd hidden layer
#nn_2.add(tf.keras.layers.Dense(units=32, activation='relu'))
nn_2.add(Dense(units=64, activation='tanh'))

# fourth Layer
nn_2.add(Dense(units=32, activation='tanh'))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
nn_2.add(Dense(units=1, activation='sigmoid'))

# Dropout Regularization
# nn_2.add(Dropout(0.2))

# Check the structure of the model
#nn_2.summary()

In [21]:
# Custome Learning rate
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [22]:
# Compile the model
nn_2.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
#prevent overfitting
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, mode='max', restore_best_weights=True)

In [24]:
#traind de model
history = nn_2.fit(X_train_scaled, y_train,
                       validation_split=0.2,
                       epochs=200,
                       batch_size=32,
                       callbacks=[early_stopping],
                       verbose=1
                   )

Epoch 1/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.6870 - loss: 0.6299 - val_accuracy: 0.7258 - val_loss: 0.5873
Epoch 2/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7174 - loss: 0.5905 - val_accuracy: 0.7300 - val_loss: 0.5807
Epoch 3/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7179 - loss: 0.5821 - val_accuracy: 0.7276 - val_loss: 0.5743
Epoch 4/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7082 - loss: 0.5942 - val_accuracy: 0.7336 - val_loss: 0.5719
Epoch 5/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7197 - loss: 0.5834 - val_accuracy: 0.7352 - val_loss: 0.5652
Epoch 6/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7250 - loss: 0.5747 - val_accuracy: 0.7278 - val_loss: 0.5740
Epoch 7/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7191 - loss: 0.5773 - val_accuracy: 0.7334 - val_loss: 0.5681
Epoch 8/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7225 - loss: 0.5730 - val_accu

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Optimized Loss: {model_loss}, Optimized Accuracy: {model_accuracy}")

215/215 - 0s - 1ms/step - accuracy: 0.7229 - loss: 0.5778
Optimized Loss: 0.5777796506881714, Optimized Accuracy: 0.722886323928833
